# Cosine Regression on the Sphere with Polynomial SGD
This notebook evaluates SGD performance on data generated from the spherical cosine model:
$$
y = \cos(\langle x, u \rangle) + \varepsilon
$$
where $ x \in \mathbb{S}^{d-1} \subset \mathbb{R}^d $. We examine the generalization and convergence behavior of fixed, halving, and diminishing stepsize strategies when fitting polynomial models to this data.